In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, udf, concat_ws
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType

import pandas as pd
import numpy as np
import unicodedata
import re

In [2]:
spark = SparkSession.builder \
        .appName("Silver_BusIds") \
        .master("local[*]") \
        .getOrCreate()

In [4]:
bus_ids = spark.read.csv("../raw/bus_company_ids.csv", header=True)

bus_ids.show(10)

+--------------------+----------+
|            Bus_Name|Company_ID|
+--------------------+----------+
|   Vũ Linh limousine|       768|
|             Ba Châu|     13078|
|            Văn Lang|     11826|
|            Trí Nhân|     12911|
|           Tuấn Hiệp|     11899|
|Thiện Thành Limou...|     21843|
|                 Hảo|     12797|
|  Trí Nhân Limousine|     12911|
|               Hà My|     12507|
|           Toàn Khải|     46289|
+--------------------+----------+
only showing top 10 rows



In [ ]:
def remove_accents(text):
    if text is None:
        return "No Info"
    # Thay thế riêng biệt chữ "đ" và "Đ" trước khi chuẩn hóa
    text = text.replace("đ", "d").replace("Đ", "D")
    # Chuẩn hóa và loại bỏ dấu tiếng Việt
    text = unicodedata.normalize('NFKD', text).encode('ASCII', 'ignore').decode('utf-8')
    # Thay thế các ký tự đặc biệt (ngoại trừ chữ, số) bằng dấu gạch ngang "-"
    text = re.sub(r"[^a-zA-Z0-9]+", "-", text)
    # Xóa dấu gạch ngang ở đầu và cuối nếu có
    return text.strip("-").lower()

remove_accents_udf = udf(remove_accents, StringType())

bus_ids = bus_ids.withColumn("Bus_Id", concat_ws("_", remove_accents_udf(col("Bus_Name")), col("Company_Id"))).select("Bus_Id", "Bus_Name", "Company_Id")
bus_ids.show()

In [6]:
spark.stop()